In [1]:
import requests
import random
import string
import psutil
import json


port = None
fn_index = None

#the server is http because speech to text js microphone needs http
cert_path =  "/appdata/RAG/trt-llm-rag-windows-ChatRTX_0.3/certs/servercert.pem"
key_path =  "/appdata/RAG/trt-llm-rag-windows-ChatRTX_0.3/certs/serverkey.pem"
ca_bundle = "/appdata/env_nvd_rag/Library/ssl/cacert.pem"

def find_chat_with_rtx_port():
    global port
    connections = psutil.net_connections(kind='inet')
    for host in connections:
        try:
            if host.pid:
                process = psutil.Process(host.pid)

                if "chatrtx" in process.exe().lower():
                    test_port = host.laddr.port
                    url = f"http://host.docker.internal:{test_port}/queue/join"

                    response = requests.post(url, data="", timeout=0.1, cert=(cert_path, key_path), verify=ca_bundle)
                    if response.status_code == 422:
                        port = test_port
                        return
        except:
            pass

def join_queue(session_hash, set_fn_index, port, chatdata):
    #fn_indexes are some gradio generated indexes from rag/trt/ui/user_interface.py
    python_object = {
        "data": chatdata,
        "event_data": None,
        "fn_index": set_fn_index,
        "session_hash": session_hash
    }
    json_string = json.dumps(python_object)

    url = f"http://host.docker.internal:{port}/queue/join"
    response = requests.post(url, data=json_string, cert=(cert_path, key_path), verify=ca_bundle)
    # print("Join Queue Response:", response)

def listen_for_updates(session_hash, port):
    url = f"http://host.docker.internal:{port}/queue/data?session_hash={session_hash}"

    response = requests.get(url, stream=True, cert=(cert_path, key_path), verify=ca_bundle)
    # print("Listen Response:", response)
    try:
        for line in response.iter_lines():
            if line:
                    data = json.loads(line[5:])
                    # if data['msg'] == 'process_generating':
                    #     print(data['output']['data'][0][0][1])
                    if data['msg'] == 'process_completed':
                        return data['output']['data'][0][0][1]
    except Exception as e:
        pass
    return ""

def auto_find_fn_index(port):
    global fn_index

    print("Searching for llm streamed completion function. Takes about 30 seconds.")
    session_hash = ''.join(random.choices(string.ascii_letters + string.digits, k=10))
    chatdata = [[["write a comma", None]], None]
    for i in range(10, 1000):
        join_queue(session_hash, i, port, chatdata)
        res = listen_for_updates(session_hash, port)
        if res:
            fn_index = i
            return
    raise Exception("Failed to find fn_index")

def send_message(message, port=port):
    global fn_index

    if not port:
        find_chat_with_rtx_port()
    if not port:
        raise Exception("Failed to find a server port for 'Chat with RTX'. Ensure the server is running.")
    if not fn_index:
        #comment this line
        auto_find_fn_index(port)
        print("To make initialization instant hardcode:\nfn_index =", fn_index)
        # fn_index = 

    session_hash = ''.join(random.choices(string.ascii_letters + string.digits, k=10))

    chatdata = [[[message, None]], None]
    join_queue(session_hash, fn_index, port, chatdata)
    return listen_for_updates(session_hash, port)

In [2]:
import os
from IPython.display import Markdown, display
import logging
import sys
from sqlalchemy.engine import URL
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    text
)
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core import SQLDatabase
from llama_index.core.retrievers import NLSQLRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

server = 'host.docker.internal'
database = 'llm_data'
username = 'sa'
password = 'Zantut123@'
driver = '{ODBC Driver 18 for SQL Server}'

# Connect to the SQL Server database
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes'
connection_url = URL.create(
    "mssql+pyodbc", 
    query={"odbc_connect": conn_str}
)
engine = create_engine(connection_url)

In [3]:
def get_db_metadata():
    metadata_obj = MetaData()
    metadata_obj.reflect(bind=engine)
    return metadata_obj.tables

def extract_query(sql_string):
    """
    Extracts the first SELECT statement from a given SQL string until the first semicolon `;`.

    Args:
        sql_string (str): The SQL string.

    Returns:
        str: The extracted SELECT statement.
    """
    # Find the index of the first occurrence of `SELECT`
    select_index = sql_string.find('SELECT')
    sc_index = sql_string.find(';')
    select_query = sql_string[select_index:sc_index].replace('\n',' ')

    return select_query.strip()

def query(response):
    with engine.connect() as connection:
        
        result = connection.execute(text(response))

        # Initialize an empty list to store the rows
        rows_list = []

        # Fetch all rows and append them to the list
        for row in result.fetchall():
            rows_list.append(row)

    # The connection will be automatically closed after exiting the 'with' block

    # Print the list of rows
    return(rows_list)

In [4]:
query_prefix_1='Dado o banco de dados: \n '
query_prefix_2='\nForneça uma query para MSSQL que responda a seguinte pergunta: '
query_prefix_3='Dada a pergunta: '
query_prefix_4=' e com a seguinte resposta: \n '
query_suffix_1= '\nuse CAST para operacoes, termine a query com ;'
query_suffix_2=' forneca um texto de resposta estruturado. forneca apenas o texto, sem nada a mais'

In [5]:
PORT=41657

In [6]:
response = send_message("Ola! voce sabe sql poderia me ajudar com umas queries?",port=PORT)
print("Response :", response)

Searching for llm streamed completion function. Takes about 30 seconds.
To make initialization instant hardcode:
fn_index = 86
Response : Sim, eu posso ajudar com queries SQL. Por favor, dize-me o que precisas.


In [7]:
query_str = "qual a soma de trabalhadores?"
db_metadata = get_db_metadata()
response = send_message(query_prefix_1+str(db_metadata).replace('FacadeDict','')+query_prefix_2+query_str+query_suffix_1,port=PORT)
#print(response)
query_str = extract_query(response)
query_result=str(query(query_str))
#print(query_result)
response2 = send_message(query_prefix_3+query_str+query_prefix_4+query_result+query_suffix_2,port=PORT)
print(response2)
print('A query utilizada foi a seguinte:\n',query_str)

O texto de resposta estruturado seria:

"The total number of workers in the PNAD_TRIMESTRAL table is 1,391,785."
A query utilizada foi a seguinte:
 SELECT SUM(CAST(vl_trabalhadores AS FLOAT)) AS total_workers FROM PNAD_TRIMESTRAL


In [8]:
query_str = "qual a maior temperatura registrada?"
db_metadata = get_db_metadata()
response = send_message(query_prefix_1+str(db_metadata).replace('FacadeDict','')+query_prefix_2+query_str+query_suffix_1,port=PORT)
#print(response)
query_str = extract_query(response)
query_result=str(query(query_str))
#print(query_result)
response2 = send_message(query_prefix_3+query_str+query_prefix_4+query_result+query_suffix_2,port=PORT)
print(response2)
print('A query utilizada foi a seguinte:\n',query_str)

O resultado da consulta SQL "SELECT MAX(CAST(temperature AS FLOAT)) AS max\_temperature FROM dht11\_room\_temperature" é:

(34.0,)

O valor máximo de temperatura registrado no banco de dados é 34.0.
A query utilizada foi a seguinte:
 SELECT MAX(CAST(temperature AS FLOAT)) AS max_temperature FROM dht11_room_temperature
